In [1]:
#IMPORTAMOS LIBRERÍA CSV PARA CARGAR ARCHIVOS CSV
import csv
#IMPORTAMOS LA LIBRERÍA PARA HACER USO DE LOGARITMO EN EL EJERCICIO 3
import math 
#DE LA LIBRERÍA PATHLIB IMPORTAMOS LA FUNCIÓN PATH PARA DEFINIR UNA RUTA HACIA NUESTROS ARCHIVOS
from pathlib import Path

#DEFINIMOS LA RUTA DE DONDE SE VAN A CARGAR LOS ARCHIVOS CSV CON LOS RUN CORRESPONDIENTES
run_basepath = Path("/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/")

In [2]:
#DEFINIMOS COLORES PARA LOS TEXTOS CON CODIGO ANSI ESCAPE EN FORMATO OCTAL Y DE COLORES EN PRIMER PLANO
#https://gist.github.com/fnky/458719343aabd01cfb17a3a4f7296797
class Colors:
    BLACK = '\033[90m'
    WHITE= '\033[97m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    CYAN = '\033[36m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    ENDC = '\033[0m'

#DEFINIMOS LA CLASE QUE REPRESENTA LA INFORMACIÓN DE UN GEN 
class Gene:
    def __init__(self, gen_symbol, expression):
        #GEN SYMBOL DEL GEN(ES DECIR, ID DE LA BASE DE DATOS SYMBOL --> EJEMPLO: IL6)
        self.gen_symbol = gen_symbol
        #NÚMERO DE LECTURAS DE LOS GENES (EXPRESIÓN)
        self.expression = expression
    
    def __str__(self) :
        return self.gen_symbol
    
    def __repr__(self) :
        return self.gen_symbol 
        
#DEFINIMOS LA CLASE QUE REPRESENTA LA INFORMACIÓN DE UNA MUESTRA
class Sample:
    def __init__(self, sample_id):
        #ID DE LAS MUESTRAS
        self.sample_id = sample_id
        #DICCIONARIO DE GENES ASOCIADOS A CADA MUESTRA
        self.genes = {}
    
    def __str__(self) :
        return self.sample_id
    
    def __repr__(self) :
        return self.sample_id
    
#DEFINIMOS LA CLASE QUE REPRESENTA LA INFORMACIÓN DE UN RUN (PROCESO DE SECUENCIACIÓN)
class Run:
    def __init__(self, run_name):
        #NOMBRE DE CADA RUN 
        self.run_name = run_name
        #DICCIONARIO DE SAMPLES CONTENIDO EN CADA RUN Y QUE PUEDEN SER DE DOS TIPOS WT Y KO
        self.samples = {}
        #DICCINARIO QUE GUARDARÁ LAS MEDIAS DE EXPRESIÓN DE CADA GEN EN LA CONDICIÓN WT
        self.expression_mean_wt = {}
        #DICCINARIO QUE GUARDARÁ LAS MEDIAS DE EXPRESIÓN DE CADA GEN EN LA CONDICIÓN KO
        self.expression_mean_ko = {}
        
    def __str__(self) :
        return self.run_name
    
    def __repr__(self) :
        return self.run_name

In [3]:
#FUNCIÓN PARA MOSTRAR AVISOS AL USUARIO Y CONTROLAR ERRORES DE VALORES DE ENTRADA O  ERORES DE CARGA DE DATOS 
def warning(message):
    print(f"{Colors.RED}* ATENCIÓN *{Colors.ENDC} {message}")

In [4]:
#FUNCIÓN QUE UTILIZA EL MÓDULO CSV.SNIFFER PARA ADIVINAR EL DELIMITADOR DEL ARCHIVO
#USANDO UNA MUESTRA DEL MISMO ARCHIVO Y DEVUELVE EL DELIMITADOR ADIVINADO
def guess_delimiter(csv_file_path):
    with open(csv_file_path, 'r') as file:
        run_data = file.read(1024)  #LEE LOS PRIMEROS 1024 BYTES DEL ARCHIVO 
        sniffer = csv.Sniffer()
        delimiter = sniffer.sniff(run_data).delimiter
        return delimiter

In [5]:
#FUNCIÓN QUE CARGARÁ CADA RUN EN UN DICCIONARIO DE DICCIONARIOS DESDE UNA RUTA Y CON CUALQUIER DELIMITADOR
#BASICAMENTE HAREMOS UN DICCIONARIO RUN CON CLAVE EL NOMBRE DEL RUN Y VALOR EL CONTENIDO DE UN RUN
#EN CADA RUN HABRÁ OTRO DICCIONARIO CON CLAVE(ID DE LAS MUESTRAS) Y VALOR EL CONTENIDO DE LAS MUESTRAS
#EN CADA MUESTRA HABRÁ OTRO DICCIONARIO CON CLAVE (GEN_SYMBOL) Y VALOR EXPRESIÓN(NUMERO DE LECTURAS)
def load_runs(run_basepath):
    runs = {}
    index = 1
    run_name = ""
    print("Cargando archivos CSV...")
    #USAMOS RGLOB PARA BÚSQUEDA RECURSIVA EN RUTA DE DIRECTORIOS
    for file in run_basepath.rglob("*.csv"):
        print(f"{file}")
        run_name = f"RUN {index}"
        runs[run_name] = Run(run_name)
        index += int(1)
        #LLAMAMOS A LA FUNCIÓN DE SNIFFER PARA QUE ACEPTE MAS DE UN TIPO DE DELIMITADOR
        delimiter = guess_delimiter(file)
        with open(file, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter=delimiter)
            for row in csv_reader:
                if "GENE" not in row:
                    continue
                #POR CADA FILA CREAMOS DICCIONARIO DE PARES MUESTRA-GEN DISCRIMINANDO ENTRE TIPOS DE MUESTRA
                for key, expression in row.items():
                    gen = Gene(row["GENE"], None)
                    if key.startswith("WT") or key.startswith("KO"):
                        sample_id = key  #UTILIZAR DIRECTAMENTE LA CLAVE COMO SAMPLE_ID
                    else:
                        continue
                    #VAMOS INCORPORANDO A RUN EL DICCIONARIO DE MUESTRAS
                    if sample_id not in runs[run_name].samples:
                        runs[run_name].samples[sample_id] = Sample(sample_id)
                    gen.expression = expression
                    expression = int(expression)
                    #VAMOS INCORPORANDO A CADA MUESTRA EL DICCIONARIO DE GENES
                    runs[run_name].samples[sample_id].genes[gen.gen_symbol] = gen
    return runs 

In [6]:
#GUARDAMOS TODO EL CONTENIDO DE LA FUNCION LOAD_RUNS EN LA VARIABLE ESTRUCTURA_DATOS_RUNS
#AQUI TE PONGO ESTE BUCLE ANIDADO POR SI QUIERES VER TODOS LOS DICCIONARIOS DE FORMA ACCESIBLE
estructura_datos_runs = load_runs(run_basepath)
#for run in estructura_datos_runs:
    #for sample in estructura_datos_runs[run].samples :
        #for gen in estructura_datos_runs[run].samples[sample].genes :
            #print(estructura_datos_runs[run].samples[sample].genes[gen].expression)

Cargando archivos CSV...
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN1/experimento1.csv
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN2/experimento2.csv
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN3/experimento3.csv
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN4/experimento4.csv


In [7]:
#FUNCIÓN QUE NOS CUENTIFICA CUANTAS MUESTRAS DE CADA TIPO HAY EN CADA UNO RUN(ARCHIVOS CSV) 
#Y TAMBIÉN NOS CUANTIFICA LOS GENES SECUENCIADOS EN CADA RUN Y LOS MUESTRA POR PANTALLA
def opcion1(runs):
    print(f"{Colors.YELLOW}\nListado de RUNs{Colors.ENDC}")
    print(f"{Colors.BLACK}-----------------------------------------------------------{Colors.ENDC}")
    #ENUMERAMOS CADA PAR CLAVE-VALOR DE NUESTRO DICCIONARIO RUN 
    for i, (run_name, run) in enumerate(runs.items(), start=1):
        wt_samples_count = 0
        ko_samples_count = 0
        gene_count = 0
        #CONTAMOS EL NÚMERO DE MUESTRAS DE CADA TIPO CON LA FUNCIÓN STARTSWITH Y LO GUARDAMOS EN VARIABLES
        for sample in run.samples:
            if sample.startswith("WT"):
                wt_samples_count += 1
            elif sample.startswith("KO"):
                ko_samples_count += 1
            #CONTAMOS TAMBIEN EL NÚMERO DE GENES CONTENIDO EN CADA UNO DE LOS DICCIONARIOS DE GENES QUE HAY EN CADA MUESTRA 
            gene_count += len(run.samples[sample].genes)
        #CON EL RECUENTO DE TODAS LAS MUESTRAS Y GENES EN TODOS LOS DICCIONARIOS --> CALCULAMOS GEN POR MUESTRA
        #YA QUE EN CADA MUESTRA TENEMOS LAS MISMAS CLAVES PARA EL DICCIONARIO GENES AUNQUE DISTINTO VALOR
        if wt_samples_count + ko_samples_count != 0:
            gene_per_sample = int(gene_count / (wt_samples_count + ko_samples_count))
            #POR ÚLTIMO IMPRIMIMOS CON CADENAS FORMATEADAS QUE NOS PERMITAN IMPRIMIR EL CONTENIDO DE VARIABLES EN EL PRINT
            print("En el RUN {} se han secuenciado: {} muestras de tipo WT y {} muestras de tipo KO, "
                 "con un total de {} genes por muestra".format(i, wt_samples_count, ko_samples_count, gene_per_sample))
        #CONTROL DE ERRORES DE CARGA DE DATOS 
        else:
            warning("No se han cargado los datos")


In [8]:
opcion1(estructura_datos_runs)


Listado de RUNs
-----------------------------------------------------------
En el RUN 1 se han secuenciado: 3 muestras de tipo WT y 4 muestras de tipo KO, con un total de 485 genes por muestra
En el RUN 2 se han secuenciado: 6 muestras de tipo WT y 4 muestras de tipo KO, con un total de 485 genes por muestra
En el RUN 3 se han secuenciado: 4 muestras de tipo WT y 5 muestras de tipo KO, con un total de 485 genes por muestra
En el RUN 4 se han secuenciado: 2 muestras de tipo WT y 4 muestras de tipo KO, con un total de 485 genes por muestra


In [9]:
#FUNCION QUE NOS CUENTA EL NUMERO DE MUESTRAS DE CADA TIPO SECUENCIADAS EN TOTAL EN TODOS LOS RUN
def opcion2(runs):
    print(f"{Colors.YELLOW}\nTotal de muestras cargadas{Colors.ENDC}")
    print(f"{Colors.BLACK}-----------------------------------------------------------{Colors.ENDC}")
    wt_count = 0
    ko_count = 0
    #ACCEDEMOS A NUESTRO DICCIONARIO RUNS, QUE TIENE LOS DICCIONARIOS DE SAMPLES Y CONTAMOS CLAVES DE ESE DICCIONARIO
    #DISCRIMINANDO SI ES WT O KO CON LA FUNCIÓN STARTSWITH Y SUMANDO Y AGREGANDO EN DOS VARIABLES DISTINTAS
    for run_name, run in runs.items():
        for sample_name, sample in run.samples.items():
            if sample_name.startswith("WT"):
                wt_count += 1
            elif sample_name.startswith("KO"):
                ko_count += 1
    #IMPRIMIMOS CON CADENA FORMATEADA DE NUEVO EL CONTENIDO DE LAS VARIABLES DE CONTEO
    print(f"Total de muestras de tipo WT: {wt_count}")
    print(f"Total de muestras de tipo KO: {ko_count}")

In [10]:
opcion2(estructura_datos_runs)


Total de muestras cargadas
-----------------------------------------------------------
Total de muestras de tipo WT: 15
Total de muestras de tipo KO: 17


In [11]:
#FUNCIÓN QUE ACTÚA DE APOYO A POSTERIORES FUNCIONES CALCULANDO LA MEDIA DE VALORES DE UNA LISTA DADA
#BÁSICAMENTE RESCATA UN DICCIONARIO FORMADO POR CLAVE:LISTA, CALCULA MEDIA DE LA LISTA Y DEVUELVE
#UN DICCIONARIO CON LA MISMA CLAVE QUE EL PRIMERO Y VALOR LA MEDIA DE LA LISTA
def media_dict(diccionario, lista):
    mean_diccionario = {}
    for gene, lista in diccionario.items():
            if lista:
                mean = round((sum(lista) / len(lista)),2)
                if gene not in mean_diccionario:
                    mean_diccionario[gene] = mean
    return mean_diccionario

In [12]:
#FUNCIÓN DE APOYO DE FUNCIONES POSTERIORES --> BÚSQUEDA DE CLAVES DE DICCIONARIO CORRESPONDIENTES A LOS PRIMEROS 1O VALORES
#ORDENADOS DE UNA LISTA Y ESA LISTA ACTÚA COMO VALOR EN EL DICCIONARIO
def clave_valor_top10(diccionario, lista):
    i = 1
    while i < 10:
        for v in lista: 
            for clave, valor in diccionario.items():
                if v == valor:
                    print(f'El gen en posición nº {i} es: {clave} -> {valor}')
                    i += 1
                if i > 10:
                    break       
    return 

In [13]:
#FUNCIÓN QUE MUESTRA AL USUARIO CUANDO LO PIDE INFORMACIÓN CONCRETA DE CADA RUN INDEPENDIENTEMENTE
#EL RUN ES PROPORCIONADO COMO ARGUMENTO DE LA FUNCIÓN
def resumen_run(runs, run_name):
    if run_name in runs:
        #RUN_NAME ES SELECCIONADO EN LA FUNCIÓN MAIN
        print(f"{Colors.YELLOW}\nResumen de expresión {run_name}{Colors.ENDC}")
        print(f"{Colors.BLACK}----------------------------------{Colors.ENDC}")
        #CONTROLAMOS UN POSIBLE ERROR DE ENTRADA SI SE SELECCIONA RUN 2
        expression_gene_all = {}
        expression_gene_wt = {}
        expression_gene_ko = {}
        #VAMOS A CREAR 3 DICCIONARIOS DONDE CLAVE=GEN Y VALOR=LISTA DE COUNTS 
        #DE CADA TIPO DE MUESTRA O DEL COJUNTO DE AMBAS
        for sample_id, sample in runs[run_name].samples.items():
            for gene in sample.genes.values():
                #SELECCIONAMOS COUNT DE CADA GEN
                expression_value = int(gene.expression)
                #INICIALIZAMOS LISTA E INCORPORAMOS A DICCIONARIO LAS DE MUESTRAS TIPO WT
                if sample_id.startswith("WT"):
                    if not gene.gen_symbol in expression_gene_wt:
                        wt_values_list = []
                    else:
                        wt_values_list = expression_gene_wt[gene.gen_symbol]
                    wt_values_list.append(expression_value)
                    expression_gene_wt[gene.gen_symbol] = wt_values_list
                #INICIALIZAMOS LISTA E INCORPORAMOS A DICCIONARIO LAS DE MUESTRAS TIPO KO           
                elif sample_id.startswith("KO"):
                    if not gene.gen_symbol in expression_gene_ko:
                        ko_values_list = []
                    else:
                        ko_values_list = expression_gene_ko[gene.gen_symbol]
                    ko_values_list.append(expression_value)
                    expression_gene_ko[gene.gen_symbol] = ko_values_list
                #INICIALIZAMOS LISTA E INCORPORAMOS A DICCIONARIO LAS DE TODAS LAS MUESTRAS       
                if not gene.gen_symbol in expression_gene_all:
                    all_values_list = []
                else:
                    all_values_list = expression_gene_all[gene.gen_symbol]
                all_values_list.append(expression_value)
                expression_gene_all[gene.gen_symbol] = all_values_list
            
        #NOS APOYAMOS EN LA FUNCION MEDIA_DICT PARA HACER LA MEDIA DE LAS LISTAS
        #Y GUARDAMOS DICCIONARIO DEVUELTO POR LA FUNCIÓN EN UNA VARIABLE USADA EN EL SIGUIENTE PASO
        expression_mean_wt = media_dict(expression_gene_wt, wt_values_list)
        expression_mean_ko = media_dict(expression_gene_ko, ko_values_list)
        expression_mean_all = media_dict(expression_gene_all, all_values_list)

        #EXTRAEMOS Y LISTAMOS LOS VALORES DE LOS DICCIONARIOS PARA SU POSTERIOR ORDENACIÓN NUMÉRICA
        all_expression = list(expression_mean_all.values())
        wt_expression = list(expression_mean_wt.values())
        ko_expression = list(expression_mean_ko.values())
        
        #ORDENAMOS LAS 3 LISTAS DE VALORES DE COUNTS NUMÉRICAMENTE
        all_expression.sort()
        wt_expression.sort()
        ko_expression.sort()
            
        #REORDENACIÓN Y SELECCIÓN DE LOS PRIMEROS 10 VALORES "MÁS" EXPRESADOS EN CADA LISTA -> ORDEN DESCENDENTE
        top10_genes_all = sorted(all_expression, reverse=True)[:10]
        top10_genes_wt = sorted(wt_expression, reverse=True)[:10]
        top10_genes_ko = sorted(ko_expression,reverse=True)[:10]
        #SELECCIÓN DE LOS PRIMEROS 10 VALORES "MENOS" EXPRESADOS EN CADA LISTA -> ORDEN ASCENDENTE
        less10_genes_all = all_expression[:10]
        less10_genes_wt = wt_expression[:10]
        less10_genes_ko = ko_expression[:10]
            
        #DEVOLVEMOS A CADA VALOR DE LISTA SU CLAVE CORRESPONDIENTE EN LOS DICCIONARIOS
        #ES DECIR, ATRIBUIMOS CADA VALOR MEDIO DE COUNT A SU SYMBOL(GEN) CORRESPONDIENTE Y LO MOSTRAMOS
        print(f"{Colors.GREEN}Los 10 genes más expresados en su valor medio en todas las muestras:{Colors.ENDC}")
        print(f"{Colors.BLACK}---------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_all, top10_genes_all)
        print("                                                                                                  ")
        print(f"{Colors.GREEN}Los 10 genes más expresados en su valor medio en las muestras tipo WT:{Colors.ENDC}")
        print(f"{Colors.BLACK}----------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_wt, top10_genes_wt)
        print("                                                                                                  ")
        print(f"{Colors.GREEN}Los 10 genes más expresados en su valor medio en las muestras tipo KO:{Colors.ENDC}")
        print(f"{Colors.BLACK}----------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_ko, top10_genes_ko)
        print("                                                                                                  ")
        print("=========================================================================================")
        print("                                                                                                  ")
        print(f"{Colors.RED}Los 10 genes menos expresados en su valor medio en todas las muestras:{Colors.ENDC}")
        print(f"{Colors.BLACK}----------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_all, less10_genes_all)
        print("                                                                                                  ")
        print(f"{Colors.RED}Los 10 genes menos expresados en su valor medio en las muestras tipo WT:{Colors.ENDC}")
        print(f"{Colors.BLACK}----------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_wt, less10_genes_wt)
        print("                                                                                                  ")
        print(f"{Colors.RED}Los 10 genes menos expresados en su valor medio en las muestras tipo KO:{Colors.ENDC}")
        print(f"{Colors.BLACK}----------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_ko, less10_genes_ko)
        
    #CONTROLAMOS POSIBLE ERROR POR SI EL USUARIO PIDE UN RUN_NAME NO PRESENTE EN LA CARPETA DE DIRCTORIOS              
    else :
        warning("RUN no registrado. No se han cargado datos. Pruebe un número de la lista de RUNs")


In [14]:
#DEFINIMOS MISMA FUNCIÓN QUE LA ANTERIOR DONDE RESUMIMOS TOMANDO COMO ARGUMENTO TODOS LOS RUN EN CONJUNTO 
#HE PROBADO MUCHOS CAMBIOS PARA AHORRARME ESTA FUNCIÓN Y/O REDUCIR ALGO DE CÓDIGO
#PERO EN TODOS ESOS CAMBIOS ME CAMBIABA EL RESULTADO
def resumen_global(runs):
    if len(runs) > 0:
        print(f"{Colors.YELLOW}\nResumen global de expresión{Colors.ENDC}")
        print(f"{Colors.BLACK}------------------------------{Colors.ENDC}")
        
        #INICIALIZAMOS 3 DICCIONARIOS CON CLAVE -> GEN Y VALOR -> LISTA DE COUNTS EN CADA MUESTRA
        expression_exp_all = {}
        expression_exp_wt = {}
        expression_exp_ko = {}
        
        #SELECCIONAMOS COUNTS EN TODOS LOS GENES DE TODAS LAS MUESTRAS DE TODOS LOS RUN 
        #Y GUARDAMOS EN 3 LISTAS DISTINTAS DISCRIMINANDO POR TIPO DE MUESTRA O LOS DOS TIPOS JUNTOS
        for run_name, run in runs.items():
            for sample_id, sample in run.samples.items():
                for gene in sample.genes.values():
                    expression_value_exp = int(gene.expression)
                    if sample_id.startswith("WT"):
                        if gene.gen_symbol not in expression_exp_wt:
                            values_exp_wt = []
                        else:
                            values_exp_wt = expression_exp_wt[gene.gen_symbol]
                        values_exp_wt.append(expression_value_exp)
                        expression_exp_wt[gene.gen_symbol] = values_exp_wt
                    elif sample_id.startswith("KO"):
                        if gene.gen_symbol not in expression_exp_ko:
                            values_exp_ko = []
                        else:
                            values_exp_ko = expression_exp_ko[gene.gen_symbol]
                        values_exp_ko.append(expression_value_exp)
                        expression_exp_ko[gene.gen_symbol] = values_exp_ko
                    
                    if not gene.gen_symbol in expression_exp_all:
                            values_exp_all = []
                    else:
                        values_exp_all = expression_exp_all[gene.gen_symbol]
                    values_exp_all.append(expression_value_exp)
                    expression_exp_all[gene.gen_symbol] = values_exp_all
                
        #NOS APOYAMOS DE NUEVO EN FUNDIÓN DE CÁLCULO DE LA MEDIA
        expression_mean_wt_exp = media_dict(expression_exp_wt, values_exp_wt)
        expression_mean_ko_exp = media_dict(expression_exp_ko, values_exp_ko)
        expression_mean_all_exp = media_dict(expression_exp_all, values_exp_all)

        #LISTAMOS LOS VALORES DE LOS DICCIONARIOS CON LA FUNCIÓN LIST
        all_exp_mean = list(expression_mean_all_exp.values())
        wt_exp_mean = list(expression_mean_wt_exp.values())
        ko_exp_mean = list(expression_mean_ko_exp.values())
        
        #ORDENAMOS LAS LISTAS NUMÉRICAMENTE
        all_exp_mean.sort()
        wt_exp_mean.sort()
        ko_exp_mean.sort()
        
        #REORDENACIÓN Y SELECCIÓN DE LOS PRIMEROS 10 VALORES "MÁS" EXPRESADOS EN CADA LISTA -> ORDEN DESCENDENTE
        top10_genes_all_exp = sorted(all_exp_mean, reverse=True)[:10]
        top10_genes_wt_exp = sorted(wt_exp_mean, reverse=True)[:10]
        top10_genes_ko_exp = sorted(ko_exp_mean,reverse=True)[:10]
        #SELECCIÓN DE LOS PRIMEROS 10 VALORES "MENOS" EXPRESADOS EN CADA LISTA -> ORDEN ASCENDENTE
        less10_genes_all_exp = all_exp_mean[:10]
        less10_genes_wt_exp = wt_exp_mean[:10]
        less10_genes_ko_exp = ko_exp_mean[:10]
        
        #DEVOLVEMOS A CADA VALOR DE LISTA SU CLAVE CORRESPONDIENTE EN LOS DICCIONARIOS
        print(f"{Colors.GREEN}Los 10 genes más expresados en su valor medio en todas las muestras de todos los experimentos:{Colors.ENDC}")
        print(f"{Colors.BLACK}-----------------------------------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_all_exp, top10_genes_all_exp)
        print("                                                                                                                            ")
        print(f"{Colors.GREEN}Los 10 genes más expresados en su valor medio en las muestras tipo WT de todos los experimentos:{Colors.ENDC}")
        print(f"{Colors.BLACK}------------------------------------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_wt_exp, top10_genes_wt_exp)
        print("                                                                                                                            ")
        print(f"{Colors.GREEN}Los 10 genes más expresados en su valor medio en las muestras tipo KO de todos los experimentos:{Colors.ENDC}")
        print(f"{Colors.BLACK}------------------------------------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_ko_exp, top10_genes_ko_exp)
        print("                                                                                                                            ")
        print("===================================================================================================")
        print("                                                                                                                            ")
        print(f"{Colors.RED}Los 10 genes menos expresados en su valor medio en todas las muestras de todos los experimentos:{Colors.ENDC}")
        print(f"{Colors.BLACK}-------------------------------------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_all_exp, less10_genes_all_exp)
        print("                                                                                                                            ")
        print(f"{Colors.RED}Los 10 genes menos expresados en su valor medio en las muestras tipo WT de todos los experimentos:{Colors.ENDC}")
        print(f"{Colors.BLACK}---------------------------------------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_wt_exp, less10_genes_wt_exp)
        print("                                                                                                                            ")
        print(f"{Colors.RED}Los 10 genes menos expresados en su valor medio en las muestras tipo KO de todos los experimentos:{Colors.ENDC}")
        print(f"{Colors.BLACK}---------------------------------------------------------------------------------------------------{Colors.ENDC}")
        clave_valor_top10(expression_mean_ko_exp, less10_genes_ko_exp)
        
        #RETORNAMOS Y GUARDAMOS EN CLASE RUN LOS DICCIONARIOS DE WT Y KO QUE NOS SERÁN ÚTILES EN LA SIGUIENTE FUNCIÓN
        return expression_mean_wt_exp, expression_mean_ko_exp
    #CONTROLAMOS ERROR DE CARGA DE DATOS EN ESTA FUNCIÓN
    else :
        warning("No se han cargado datos") 
        return None
    

In [15]:
resumen_all_experiments = resumen_global(estructura_datos_runs)


Resumen global de expresión
------------------------------
Los 10 genes más expresados en su valor medio en todas las muestras de todos los experimentos:
-----------------------------------------------------------------------------------------------
El gen en posición nº 1 es: MANEA -> 648.25
El gen en posición nº 2 es: NOD2 -> 641.53
El gen en posición nº 3 es: NUCB1 -> 632.34
El gen en posición nº 4 es: IL9 -> 620.62
El gen en posición nº 5 es: BANK1 -> 612.75
El gen en posición nº 6 es: NPEPPS -> 610.69
El gen en posición nº 7 es: JUNB -> 609.19
El gen en posición nº 8 es: CD1A -> 609.09
El gen en posición nº 9 es: CFTR -> 608.31
El gen en posición nº 10 es: PLIN1 -> 605.59
                                                                                                                            
Los 10 genes más expresados en su valor medio en las muestras tipo WT de todos los experimentos:
-------------------------------------------------------------------------------------------

In [16]:
#FUNCION PARA CALCULAR EL LOG FOLD CHANGE Y DEVOLVER GENES SOBREEXPRESADOS E INFRAEXPRESADOS
def calculate_LFC(resumen_all_experiments, LFC_value):
    #USAMOS COMO ARGUMENTO DE LA FUNCIÓN LOS DICCIONARIOS DE LA ANTERIOR Y UN VALOR DE LFC APORTADO POR EL USUARIO
    print(f"{Colors.YELLOW}\nExpresión diferencial en base a Log Fold Change (|LFC| -> {LFC_value}){Colors.ENDC}")
    print(f"{Colors.BLACK}---------------------------------------------------------------------------------{Colors.ENDC}")
    
    #INCORPORAMOS LA VARIABLE QUE CONTIENE LOS DOS DICCIONARIOS USADOS COMO PUNTO DE PARTIDA
    if resumen_all_experiments is not None:
        expression_mean_wt_exp, expression_mean_ko_exp = resumen_all_experiments
        
        #INICIALIZAMOS DOS NUEVOS DICCIONARIOS QUE CONTENDRÁN CLAVE -> GEN , VALOR -> LFC DEL GEN
        #UN DICCIONARIO PARA LOS GENES SOBRE EXPRESADOS Y OTRO PARA LOS INFRAEXPRESADOS
        genes_sobre = {}
        genes_infra = {}
        
        #BUSCAMOS LOS VALORES MEDIOS DE COUNTS DE CADA GEN EN CADA TIPO DE MUESTRA
        #Y LOS VAMOS GUARDANDO EN DOS VARIABLES
        for gene, wt_exp in expression_mean_wt_exp.items():
            if gene in expression_mean_ko_exp:
                ko_exp = expression_mean_ko_exp[gene]
            else:
                ko_exp = 0.0
            #CONDICIONAMOS QUE NO SE PRODUZCAN DIVISIONES ENTRE 0
            if wt_exp != 0.0 and ko_exp != 0.0:
                #CALCULAMOS VALOR LFC PARA CADA GEN Y REDONDEAMOS A DOS CIFRAS DECIMALES
                lfc = round(math.log2(wt_exp / ko_exp),2)
                #AHORA SELECCIONAMOS SOLAMENTE AQUELLOS VALORES DE LFC SUPERIORES AL VALOR LFC
                #PROPORCIONADO POR EL USUARIO EN VALOR ABSOLUTO -> VALOR DE "CAMBIO DE EXPRESIÓN"
                if abs(lfc) >= abs(LFC_value):
                    #SI ES MAYOR O IGUAL QUE EL LFC FIJADO SERÁ CONSIDERADO GEN SOBREEXPRESADO
                    #Y SE GUARDARÁ EN EL DICCIONARIO CORRESPONDIENTE
                    if lfc >= LFC_value:
                        genes_sobre[gene] = lfc
                    #LO MISMO PERO PARA EL CASO DE LOS VALORES NEGATIVOS, QUE SERÁN CONSIDERADOS
                    #INFRAEXPRESADOS Y SE GUAEDARÁN COMO TAL
                    else:
                        genes_infra[gene] = lfc
        
        #SI EL VALOR PROPORCIONADO POR EL USUARIO EXCEDE A LOS VALORES REGISTRADOS SE MOSTRARÁ UN AVISO
        if not genes_sobre and not genes_infra:
            warning(f"No hay genes con un |LFC| >= {LFC_value}")
        #SI EL VALOR LFC CONTIENE REGISTROS NEGATIVOS PERO NO POSITIVOS, SE MOSTRARÁ UN AVISO
        #Y SE MOSTRARÁ LA LISTA DE GENES INFRAEXPRESADOS CORRESPONDIENTE
        elif not genes_sobre:
            print(f"{Colors.RED}Listado de genes infraexpresados{Colors.ENDC}")
            print(f"{Colors.BLACK}-------------------------------{Colors.ENDC}")
            for gene, lfc in genes_infra.items():
                print(f"Gen {gene} ----> LFC = {lfc}")
            print("                                                            ")
            warning(f"No hay genes sobreexpresados con un |LFC| >= {LFC_value}")
        #SI EL VALOR LFC CONTIENE REGISTROS POSITIVOS PERO NO NEGATIVOS, SE MOSTRARÁ UN AVISO
        #Y SE MOSTRARÁ LA LISTA DE GENES SOBREEXPRESADOS CORRESPONDIENTE
        elif not genes_infra:
            print(f"{Colors.GREEN}Listado de genes sobreexpresados{Colors.ENDC}")
            print(f"{Colors.BLACK}-------------------------------{Colors.ENDC}")
            for gene, lfc in genes_sobre.items():
                print(f"Gen {gene} ----> LFC = {lfc}")
            print("                                                            ")
            warning(f"No hay genes infraexpresados con un |LFC| >= {LFC_value}")
        #SI EL VALOR PROPORCIONADO ESTÁ DENTRO DEL RANGO DE REGISTRADOS PARA LOS GENES
        #SE MOSTRARÁ LAS CORRESPONDIENTES LISTAS DE GENES SOBRE E INFRAEXPRESADOS
        else:
            print(f"{Colors.GREEN}Listado de genes sobreexpresados{Colors.ENDC}")
            print(f"{Colors.BLACK}-------------------------------{Colors.ENDC}")
            for gene, lfc in genes_sobre.items():
                print(f"Gen {gene} ----> LFC = {lfc}")
            print("                                                            ")
            print(f"{Colors.RED}Listado de genes infraexpresados{Colors.ENDC}")
            print(f"{Colors.BLACK}-------------------------------{Colors.ENDC}")
            for gene, lfc in genes_infra.items():
                print(f"Gen {gene} ----> LFC = {lfc}")
    #CONTROLAMOS QUE LA CARGA DE DATOS SEA ADECUADA Y SI NO ES ASÍ QUE MUESTRE UN AVISO
    else:
        warning("No se han cargado datos")

In [17]:
#DEFINIMOS ESTA FUNCIÓN DE APOYO AL CONTROL DE ERRORES POR ENTRADA EN LA OPCIÓN 4 DE LA FUNCION MAIN
#YA QUE SI CONTROLAMOS POR LA PRESENCIA DEL PUNTO (".") COMO EN EL CASO DE LA FUNCIÓN RESUMEN_RUN
#NO ME PUEDE PASAR EL STR A FLOAT EN ESTE CASO
def is_decimal(numero):
    if '.' in numero:
        #CORTAMOS LA PARTE DE PUNTO Y COMPROBAMOS QUE AMBOS CARACTERES SON NUMÉRICOS
        partes = numero.split('.')
        if len(partes) == 2 and partes[0].isdigit() and partes[1].isdigit():
            return True
    return False

In [19]:
#FUNCION PRINCIPAL DE MENÚ DE OPCIONES QUE TIENE EL USUARIO Y SUBMENU PARA QUE ELIJA QUE QUIERE VER 
#NOTA:PARA QUE FUNCIONE LAS FUNCIONES SUPERIORES HAY QUE COPIARLAS Y PEGARLAS EN ESTE BLOQUE DE CODIGO
def main():
    #DEJO ESTO A PROPOSITO AQUÍ PARA QUE APAREZCA VISUALMENTE LOS ARCHIVOS QUE SE CARGAN JUNTO AL MENÚ
    estructura_datos_runs = load_runs(run_basepath)       
    #MENÚ PRINCIPAL DEL PROGRAMA CON 5 OPCIONES PARA EL USUARIO DONDE CADA OPCIÓN
    #SERÁ ARGUMENTO DE UNA FUNCIÓN ANTERIOREMENTE DEFINIDA Y LA ÚLTIMA CORTA EL BUCLE PARA SALIR
    #CONTROLAMOS POSIBLES ERRORES MOSTRANDO UN AVISO CUANDO NO SE SELECCIONA NINGUNA DE LAS OPCIONES
    while True:
        print(f"{Colors.BLUE}\nMENÚ PRINCIPAL{Colors.ENDC}")
        print(f"{Colors.BLACK}---------------------------------------------------------{Colors.ENDC}")
        print("1. Listar RUNs")
        print("2. Total de muestras cargadas")
        print("3. Resumen de expresión RUNs")
        print("4. Log fold change RUNs")
        print("5. Salir del programa")
        opcion = input("\nIndique la opción que desea ejecutar (1, 2, 3, 4 o 5): ")
        if opcion == "1":
            opcion1(estructura_datos_runs)
        elif opcion == "2":
            opcion2(estructura_datos_runs)
        elif opcion == "3":
                #SUBMENÚ PARA LAS FUNCIONES RESUMEN DONDE LA ENTRADA SE GUARDA EN LA VARIABLE
                #SPECIFIC_RUN Y ESTARÁ MUY CONTROLADA PARA EVITAR POSIBLES ERRORES
                while True:
                    print(f"\n{Colors.BLUE}SUBMENU > Resumen de expresión RUNs{Colors.ENDC}")
                    print("---------------------------------------------------------")
                    #TENDREMOS TANTOS RUN DISPONIBLES COMO ARCHIVOS CSV HAYAMOS CARGADO
                    for r in estructura_datos_runs.keys():
                        print(f"Resumen expresión {r}")
                    print("                                 ")
                    #SERÁ POSIBLE VOLVER AL MENÚ PRINCIPAL CON CARACTERES NO NUMÉRICOS 
                    #INCLUIDOS NÚMEROS DECIMALES SEPARADOS POR (,) EN VEZ DE (.)
                    print("Volver a menú principal (Pulse un carácter o cadena no numérico)")
                    specific_run = input("\nIndique el RUN que desea visualizar: ")
                    #SI NO SE PROPORCIONA UN VALOR POR LA ENTRADA SE MUESTRA LA FUNCIÓN DE RESUMEN GLOBAL
                    if specific_run == "":
                        resumen_global(estructura_datos_runs)
                    #SI EL VALOR PROPORCIONADO POR EL USUARIO ES UN NÚMERO Y CORRESPONDE A UNO DE LOS RUN
                    #QUE TENEMOS DISPONIBLES SE GUARDARÁ COMO ENTERO
                    elif specific_run.isnumeric() and specific_run <= r:
                        run_number = int(specific_run)
                        #DEPENDIENDO DE SI ES DISTINTO DE 0 O NO SE MOSTRARÁ AVISO
                        #O RESUMEN DEL RUN SELECCIONADO
                        if run_number != 0:
                            resumen_run(estructura_datos_runs, f"RUN {specific_run}")
                        elif run_number == 0:
                            warning("La opción indicada no está disponible. Pruebe de nuevo:\n")
                    #VERIFICAMOS SI ES UN NÚMERO DECIMAL Y NO SERÁ VÁLIDO COMO ARGUMENTO DE LA FUNCIÓN (MOSTRAMOS AVISO)
                    elif "." in specific_run:
                        warning("La opción indicada no está disponible. Pruebe de nuevo:\n")
                    #SI EL VALOR PROPORCIONADO ES CARÁCTER O CADENA DE CARACTERES ROMPEMOS BUCLE Y SALIMOS AL MENÚ PRINCIPAL
                    else:
                        break
                

        elif opcion == "4":
            #SUBMENÚ LOG FOLD CHANGE DONDE SE RECIBE UN VALOR UMBRAL POR PARTE DEL USUARIO
            while True:
                print(f"\n{Colors.BLUE}SUBMENU > Resumen diferencias expresion génica (LFC){Colors.ENDC}")
                print("---------------------------------------------------------")
                #VALOR SE GUARDA EN LA VARIABLE LFC_VALUE
                LFC_value = input("\nIndique un valor |LFC| que desea visualizar: ")
                print("                                 ")
                #DE IGUAL MODO QUE PARA EL RESUMEN, INCLUIMOS OPCIÓN DE VUELTA AL MENÚ PRINCIPAL
                print("Volver a menú principal (Pulse un carácter o cadena no numérico)")
                #CONTTOLAMOS QUE EL VALOR SEA DECIMAL O NUMÉRICO Y PASARÁ EL LFC_VALUE A VARIABLE TIPO FLOAT
                if is_decimal(LFC_value) or LFC_value.isnumeric(): 
                    LFC_user = float(LFC_value)
                    calculate_LFC(resumen_all_experiments, LFC_user)
                #EN CASO CONTRARIO AL ANTERIOR, SE ROMPE BUCLE, SALIMOS AL MENÚ PRINCIPAL
                else:
                    break 
        #SI SE MARCA LA OPCIÓN 5 ROMPEMOS BUCLE DE MENÚ PRINCIPAL Y SALIMOS DEL PROGRAMA
        #Y LO MOSTRAMOS CON EL ÚLTIMO PRINT
        elif opcion == "5":
                break
        #EN CASO DE QUE EL USUARIO COMETA ALGÚN TIPO DE ERROR SE MOSTRARÁ UN AVISO
        #Y SE LE DARÁ LA OPORTUNIDAD DE VOLVER A INTENTARLO
        else:
            warning("La opción indicada no está disponible")
                
    print(f"{Colors.CYAN}\nEl programa ha finalizado correctamente\n{Colors.ENDC}")
#PRIMERA FUNCIÓN EN INICIALIZARSE SERÁ ESTA    
if __name__ == "__main__":
    main()

Cargando archivos CSV...
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN1/experimento1.csv
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN2/experimento2.csv
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN3/experimento3.csv
/home/alumno15/Sistemas Bioinformáticos/Practica1/runs/RUN4/experimento4.csv

MENÚ PRINCIPAL
---------------------------------------------------------
1. Listar RUNs
2. Total de muestras cargadas
3. Resumen de expresión RUNs
4. Log fold change RUNs
5. Salir del programa



Indique la opción que desea ejecutar (1, 2, 3, 4 o 5):  5



El programa ha finalizado correctamente

